In [ ]:
from IPython.display import HTML
from matplotlib_inline.backend_inline import set_matplotlib_formats

# Use inline backend so CSS scaling works on PNG/SVG outputs
set_matplotlib_formats('png')

HTML("""
<style>
/* =======================
   Matplotlib figure sizing
   ======================= */

/* Images (PNG/retina) */
.jp-RenderedImage img,
.jp-OutputArea-output img {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
  display: block;
  box-sizing: border-box;
}

/* SVG outputs */
.jp-RenderedSVG svg,
.jp-OutputArea-output svg {
  width: 100% !important;
  height: auto !important;
}

/* ipympl / canvas-backed figures (in case %matplotlib widget is used) */
.jupyter-matplotlib canvas.mpl-canvas,
.jp-OutputArea-output canvas.mpl-canvas,
.jp-OutputArea-output canvas {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
}

/* Reset container widths */
.jp-OutputArea-output .matplotlib-figure,
.jp-RenderedHTMLCommon .matplotlib-figure,
.jp-OutputArea-output .output_png {
  width: 100% !important;
}

/* =======================
   ipywidgets styling
   ======================= */

/* Make slider labels bold */
.widget-label {
    font-weight: bold;
}

/* Style ipywidgets buttons */
.jp-OutputArea .widget-button,
button.widget-button {
    font-weight: bold;
    background-color: #1976d2;  /* blue background */
    color: white;               /* white text */
    border: none;
    border-radius: 6px;
    padding: 6px 12px;
    font-size: 14px;
    cursor: pointer;
}

/* Button hover effect */
.jp-OutputArea .widget-button:hover,
button.widget-button:hover {
    background-color: #1259a4;  /* darker blue */
}
</style>
""")

# Harmoninen värähtelijä 🎵🔄

Yksi liikkeen perusmuodoista on **tasaisin väliajoin toistuva liike**. Tätä kutsutaan myös *jaksolliseksi liikkeeksi*.  

Ajattele esimerkiksi keinussa istuvaa lasta 👦🏻🪀: hän liikkuu edestakaisin — ensin eteen, sitten taakse — ja sama kuvio toistuu yhä uudestaan.  
Toinen tuttu esimerkki on **maan pyöriminen** 🌍, joka synnyttää meille päivän ja yön vuorottelun.  

---

## Taajuus ja jakso ⏱️❤️

Jaksolliselle liikkeelle olennainen käsite on **taajuus**: kuinka usein liike toistuu tietyssä ajassa. Esimerkiksi ihmisen sydän sykkii säännöllisesti — noin 60 kertaa **minuutissa** (eli noin kerran sekunnissa ~1 Hz). Jaksojen lukumäärää sekunnissa kutsutaan *hertseiksi* (Hz), tutkija Heinrich Hertzin mukaan 📡.

---

## Kevyt mallinnus: jousi ja punnus ⚖️🌀

Tutkitaan jaksollista liikettä simulaation avulla. Kuvittele seinään kiinnitetty **jousi**, jonka päähän on laitettu **punnus**. Punnus pääsee liukumaan pöydällä lähes kitkattomasti vaakasuunnassa.  

Kun järjestelmä on levossa, punnus on **tasapainoasemassa**. Voit *vetää* tai *työntää* punnusta säätämällä simulaatiossa parametria `x₀` ja näin poikkeuttaa sen tasapainosta.

Tämä systeemi on esimerkki **harmonisesta värähtelijästä**. Sen erikoisuus on, että punnukseen kohdistuva voima on **suoraan verrannollinen poikkeamaan**: mitä enemmän venytät jousta, sitä voimakkaampi palauttava voima on 💪.

👉 *Amplitudilla* tarkoitetaan tätä suurinta poikkeamaa tasapainoasemasta.

---

## Vaimennus ja kitka 🌬️

Oikeassa maailmassa jousi ja punnus eivät värähtele ikuisesti. Niihin kohdistuu **kitkaa ja ilmanvastusta**, jotka vähitellen syövät liikkeen energiaa. Simulaatiossa voit säätää vaimennuskerrointa `ζ` ja nähdä, miten värähtely vähitellen hiipuu.

---

## Jousivakio k 🪢

Jousella on myös oma "jäykkyytensä".  
- Jos jousi on **pehmeä**, sen venyttäminen vaatii vähän voimaa.  
- Jos jousi on **jäykkä**, tarvitset enemmän voimaa punnuksen poikkeuttamiseen.  

Mitä jäykempi jousi, sitä nopeammin punnus pyrkii takaisin tasapainoon. Tämä kasvattaa myös liikkeen **taajuutta** 🔔.

---

## Nopeus ja paikka 🚀

Simulaatiossa näet myös kappaleen **paikan x(t)** ja **nopeuden v(t)**:  
- Nopeus on suurimmillaan, kun punnus kulkee tasapainoaseman läpi.  
- Poikkeaman ääripisteessä nopeus hidastuu nollaan, ja kappale kääntää suuntaa.  

Näin syntyy se edestakainen värähtely tasapainoaseman ympärillä.

---

## Energia muuttaa muotoaan 🔋⚡

Värähtelyn aikana systeemin (jousi ja punnus) energia siirtyy liike-energiasta potentiaalienergiaan ja takaisin:  
- Kun punnus kulkee tasapainoaseman ohi, kaikki energia on sen **liikkeessä**.  
- Kun punnus pysähtyy hetkeksi ääriasemaan, kaikki energia on varastoitunut **jouseen**.  

Tämä energian vuorottelu tekee harmonisesta värähtelystä kauniin ja yleisen ilmiön ✨.

---

👉 Säädä alla olevan simulaation massaa, jousivakiota, vaimennusta ja alkupoikkeamaa ja katso, miten liike, nopeus ja energiat käyttäytyvät.


In [ ]:
def build_harmonic_oscillator_sim():
    """
    Voila-ready horizontal spring–mass (harmonic oscillator) demo.

    Updates:
      - Time-series lines *grow* with time (no vertical cursor line).
      - Improved spacing: uses constrained_layout + larger figure + label padding.

    Returns
    -------
    ipywidgets.VBox
        Controls + live Matplotlib output area.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import (
        FloatSlider, IntSlider, Play, jslink, HBox, VBox, Layout,
        Checkbox, Button, Label, Output
    )
    from IPython.display import display, clear_output

    # -----------------------
    # Simulation timeline
    # -----------------------
    FPS = 60
    DURATION = 12.0
    DT = 1.0 / FPS
    N_FRAMES = int(DURATION / DT)
    t = np.arange(N_FRAMES) * DT

    # -----------------------
    # Widgets (model params + controls)
    # -----------------------
    m = FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1,
                    description="Massa m (kg)", readout_format=".2f")
    m.style.description_width = 'initial'
    k = FloatSlider(value=4.0, min=0.1, max=50.0, step=0.1,
                    description="Jousivakio k (N/m)", readout_format=".2f")
    k.style.description_width = 'initial'
    x0 = FloatSlider(value=0.25, min=-1.0, max=1.0, step=0.01,
                     description="Alkuperäinen x₀ (m)", readout_format=".2f")
    x0.style.description_width = 'initial'
    damping = FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01,
                          description="Vaimennus ζ", readout_format=".2f")
    damping.style.description_width = 'initial'
    show_trail = Checkbox(value=True, description="Näytä jälki")

    play = Play(value=0, min=0, max=N_FRAMES-1, step=1, interval=int(1000/FPS))
    frame = IntSlider(value=0, min=0, max=N_FRAMES-1, step=1, description="Frame")
    jslink((play, 'value'), (frame, 'value'))

    reset_btn = Button(description="Reset", button_style='info')
    save_btn = Button(description="Tallenna kuva", button_style='success')

    readout = Label()
    top_row = HBox([m, k, x0], layout=Layout(column_gap='12px'))
    mid_row = HBox([damping, show_trail],
                   layout=Layout(column_gap='12px', align_items='center'))
    bottom_row = HBox([play, frame, reset_btn, save_btn],
                      layout=Layout(column_gap='12px', align_items='center'))

    # -----------------------
    # Output + Figure (constrained layout to prevent overlap)
    # Layout: 1) Scene  2) x(t)  3) v(t)  4) Energies
    # -----------------------
    out = Output()
    fig, axs = plt.subplots(
        4, 1,
        figsize=(10, 10.5),
        constrained_layout=True,
        gridspec_kw=dict(height_ratios=[2.0, 1.2, 1.2, 1.6])
    )
    ax_scene, ax_x, ax_v, ax_E = axs
    plt.close(fig)  # avoid static capture

    # add a bit of label padding
    for ax in (ax_x, ax_v, ax_E):
        ax.xaxis.labelpad = 6
        ax.yaxis.labelpad = 8

    # -----------------------
    # Geometry / drawing helpers
    # -----------------------
    left_wall_x = 0.0
    x_eq = 0.8              # equilibrium center x-position of the block (plot coords)
    mass_w, mass_h = 0.12, 0.12
    y0_rect = -mass_h / 2

    def spring_coords(x_left, x_right, y=0.0, coils=14, amp=0.05):
        """Return (xs, ys) for a zig–zag spring line from x_left to x_right."""
        x_right = max(x_right, x_left + 1e-6)
        xs = np.linspace(x_left, x_right, coils * 2 + 1)
        ys = np.full_like(xs, y)
        ys[1:-1:2] += amp
        ys[2:-1:2] -= amp
        ys[0] = y
        ys[-1] = y
        return xs, ys

    # -----------------------
    # Trajectory generator
    # -----------------------
    def trajectory(m_, k_, zeta_, x0_, v0_):
        """
        Returns x_center(t), v_center(t) in plot coordinates where equilibrium is at x_eq.
        ODE: m x'' + c x' + k x = 0, with c = 2 ζ sqrt(k m).
        """
        m_ = float(m_)
        k_ = float(k_)
        zeta_ = float(zeta_)
        x0_ = float(x0_)
        v0_ = float(v0_)

        if m_ <= 0 or k_ <= 0:
            return np.full_like(t, x_eq), np.zeros_like(t)

        omega_n = np.sqrt(k_ / m_)
        if zeta_ == 0.0:
            x_phys = x0_ * np.cos(omega_n * t) + (v0_ / omega_n) * np.sin(omega_n * t)
            v_phys = -x0_ * omega_n * np.sin(omega_n * t) + v0_ * np.cos(omega_n * t)
        else:
            if zeta_ < 1.0:  # underdamped
                omega_d = omega_n * np.sqrt(1.0 - zeta_**2)
                A = x0_
                B = (v0_ + zeta_ * omega_n * x0_) / omega_d
                e = np.exp(-zeta_ * omega_n * t)
                c, s = np.cos(omega_d * t), np.sin(omega_d * t)
                x_phys = e * (A * c + B * s)
                v_phys = e * (-zeta_ * omega_n * (A * c + B * s) + (-A * omega_d * s + B * omega_d * c))
            elif zeta_ == 1.0:  # critically damped
                A = x0_
                B = v0_ + omega_n * x0_
                e = np.exp(-omega_n * t)
                x_phys = (A + B * t) * e
                v_phys = (B - omega_n * (A + B * t)) * e
            else:  # overdamped
                s1 = -omega_n * (zeta_ - np.sqrt(zeta_**2 - 1.0))
                s2 = -omega_n * (zeta_ + np.sqrt(zeta_**2 - 1.0))
                denom = (s1 - s2)
                if abs(denom) < 1e-12:
                    A = x0_
                    B = 0.0
                else:
                    A = (v0_ - s2 * x0_) / (s1 - s2)
                    B = x0_ - A
                x_phys = A * np.exp(s1 * t) + B * np.exp(s2 * t)
                v_phys = A * s1 * np.exp(s1 * t) + B * s2 * np.exp(s2 * t)

        x_center = x_eq + x_phys
        v_center = v_phys
        return x_center, v_center

    # Cached arrays
    x_traj, v_traj = trajectory(m.value, k.value, damping.value, x0.value, 0.0)

    # -----------------------
    # Main draw routine
    # -----------------------
    def _draw(idx: int):
        idx = int(np.clip(idx, 0, N_FRAMES - 1))
        nonlocal x_traj, v_traj
        x_traj, v_traj = trajectory(m.value, k.value, damping.value, x0.value, 0.0)

        xc = float(x_traj[idx])
        t_now = t[idx]
        m_, k_, zeta_ = float(m.value), float(k.value), float(damping.value)
        omega_n = np.sqrt(k_ / m_) if m_ > 0 and k_ > 0 else 0.0
        T = (2 * np.pi / omega_n) if omega_n > 0 else np.inf

        # Physical displacement relative to equilibrium for plots
        disp = x_traj - x_eq
        vel = v_traj

        # Energies
        E_p = 0.5 * k_ * disp**2
        E_k = 0.5 * m_ * vel**2
        E = E_p + E_k

        with out:
            clear_output(wait=True)

            # --- Scene (top panel) ---
            ax_scene.cla()
            ax_scene.plot([left_wall_x, left_wall_x], [-0.25, 0.25], lw=6)
            ax_scene.plot([-0.1, 1.6], [-0.25, -0.25], lw=1)  # floor
            rect = plt.Rectangle((xc - mass_w / 2, y0_rect), mass_w, mass_h)
            ax_scene.add_patch(rect)
            xs, ys = spring_coords(left_wall_x, xc - mass_w / 2, y=0.0, coils=14, amp=0.05)
            ax_scene.plot(xs, ys, lw=2)
            ax_scene.plot([x_eq, x_eq], [0.28, 0.34], lw=2)
            ax_scene.text(x_eq, 0.305, "Tasapainoasema", ha="center", va="bottom", fontsize=9)
            if show_trail.value:
                ax_scene.plot(x_traj[:idx+1], np.zeros(idx+1), lw=1.5, alpha=0.6)
            ax_scene.set_xlim(-0.1, 1.6)
            ax_scene.set_ylim(-0.3, 0.4)
            ax_scene.set_aspect('equal')
            ax_scene.set_yticks([])
            ax_scene.set_xlabel("x (m)")
            ax_scene.set_title("Jousijärjestelmä", pad=8)

            # --- Position subplot: progressive line up to current frame ---
            ax_x.cla()
            ax_x.plot(t[:idx+1], disp[:idx+1], lw=1.8, label="x(t) (m)")
            ax_x.set_xlim(0, DURATION)
            ylim = max(0.2, float(np.max(np.abs(disp[:max(1, idx+1)]))) * 1.1)
            ax_x.set_ylim(-ylim, ylim)
            ax_x.set_ylabel("x (m)")
            ax_x.grid(True, alpha=0.3)
            ax_x.legend(loc="upper right", frameon=False)
            ax_x.set_title("Paikka vs Aika", pad=8)
            ax_x.margins(x=0)

            # --- Velocity subplot: progressive line ---
            ax_v.cla()
            ax_v.plot(t[:idx+1], vel[:idx+1], lw=1.8, label="v(t) (m/s)")
            ax_v.set_xlim(0, DURATION)
            vlim = max(0.2, float(np.max(np.abs(vel[:max(1, idx+1)]))) * 1.1)
            ax_v.set_ylim(-vlim, vlim)
            ax_v.set_ylabel("v (m/s)")
            ax_v.grid(True, alpha=0.3)
            ax_v.legend(loc="upper right", frameon=False)
            ax_v.set_title("Nopeus vs Aika", pad=8)
            ax_v.margins(x=0)

            # --- Energy subplot: progressive lines ---
            ax_E.cla()
            ax_E.plot(t[:idx+1], E_p[:idx+1], lw=1.5, label="Potentiaalienergia ½kx²")
            ax_E.plot(t[:idx+1], E_k[:idx+1], lw=1.5, label="Liike-energia ½mv²")
            ax_E.plot(t[:idx+1], E[:idx+1],  lw=1.8, label="Kokonaisenergia E")
            ax_E.set_xlim(0, DURATION)
            ax_E.set_xlabel("Aika (s)")
            ax_E.set_ylabel("Energia (J)")
            ax_E.grid(True, alpha=0.3)
            ax_E.legend(loc="upper right", frameon=False)
            ax_E.set_title("Energia vs Aika", pad=8)
            ax_E.margins(x=0)

            # Readout
            readout.value = (
                f"m = {m_:.2f} kg,  k = {k_:.2f} N/m,  ζ = {zeta_:.2f},  "
                f"x₀ = {float(x0.value):.2f} m,  v₀ = 0.0 m/s    |    "
                f"ωₙ = {omega_n:.3f} rad/s,  T = {'∞' if not np.isfinite(T) else f'{T:.3f} s'},  t = {t_now:.2f} s"
            )

            display(fig)

    # -----------------------
    # Handlers
    # -----------------------
    def _reset(_):
        m.value = 1.0
        k.value = 4.0
        x0.value = 0.25
        damping.value = 0.0
        frame.value = 0
        play.value = 0
        _draw(0)

    def _save(_):
        import os
        os.makedirs('outputs', exist_ok=True)
        fname = (
            f"outputs/harmonic_oscillator_m{m.value:.2f}_k{k.value:.2f}_"
            f"x0{x0.value:.2f}_zeta{damping.value:.2f}_"
            f"frame{int(frame.value)}.png"
        )
        fig.savefig(fname, dpi=150, bbox_inches='tight')
        print(f"Tallennettu {fname}")

    reset_btn.on_click(_reset)
    save_btn.on_click(_save)

    # Redraw when params change (hold frame index)
    for w in (m, k, x0, damping, show_trail):
        w.observe(lambda _: _draw(frame.value), names='value')

    # Frame playback
    frame.observe(lambda ch: _draw(ch['new']), names='value')

    # Initial render
    _draw(frame.value)

    return VBox([top_row, mid_row, bottom_row, readout, out])

# Build & display the app in a notebook/Voila:
app = build_harmonic_oscillator_sim()
display(app)